In [1]:
import sys
import os

# Get the absolute path to the project directory
project_dir = os.path.abspath("..")

# Append the project directory to sys.path
if project_dir not in sys.path:
    sys.path.append(project_dir)

In [2]:
import sys
import os

# Get the absolute path to the project directory
project_dir = os.path.abspath("..")

# Append the project directory to sys.path
if project_dir not in sys.path:
    sys.path.append(project_dir)

import pandas as pd
import numpy as np
import os
from typing import Dict
import datetime as dt
from src.common.DataFrameTimeOperations import DataFrameTimeOperations as DOps

from src.common.AssetDataPolars import AssetDataPolars
from src.common.AssetDataService import AssetDataService
from src.common.AssetFileInOut import AssetFileInOut 
from src.featureAlchemy.FeatureTA import FeatureTA
from src.featureAlchemy.FeatureTATS import FeatureTATS


In [3]:
assets=AssetFileInOut("../src/stockGroups/bin").loadDictFromFile("group_debug")

# Convert to Polars for speedup
assetspl: Dict[str, AssetDataPolars] = {}
for ticker, asset in assets.items():
    assetspl[ticker] = AssetDataService.to_polars(asset)

In [4]:
asset = assetspl["EXC"]
params = {
    'timesteps': 50,
    'lagList': [1, 2, 3, 5, 9],
}
workdays = pd.date_range(start=dt.date(2020, 1, 1), end=dt.date(2020, 12, 31), freq='B')
dates = [date.date() for date in workdays]
startDate = pd.Timestamp(2019, 7, 15, tz="UTC").date()
endDate = pd.Timestamp(2021, 7, 15, tz="UTC").date()
ffd = FeatureTA(asset, startDate, endDate, params=params)

In [5]:
featureNames = ffd.getFeatureNames()
features = ffd.apply(dates)

date4 = pd.Timestamp(2019, 7, 15, tz = "UTC").date()
features4 = ffd.apply([date4])
date5 = pd.Timestamp(2019, 7, 16, tz = "UTC").date()
features5 = ffd.apply([date5])

print(features4[0,0])
print(features4[0,85])

idx = DOps(asset.shareprice).getNextLowerOrEqualIndex(date4)
openprice = asset.shareprice["Open"].item(idx)
openpricelast = asset.shareprice["Open"].item(idx-1)
closeprice = asset.shareprice["Close"].item(idx)
closepricelast = asset.shareprice["Close"].item(idx-1)
print(openprice/closeprice)
print(openpricelast/closeprice)
print(asset.shareprice["Date"].item(idx))

1.0026596
1.0151391
1.002659574468085
1.015139116202946
2019-07-15


In [7]:
ffdts = FeatureTATS(asset, startDate, endDate, params=params)
featureNamesTS = ffdts.getFeatureNames()
featuresTS1 = ffdts.apply(dates)
openidx = featureNamesTS.index("FeatureTA_Open")

featureTS0 = featuresTS1[0]
featureTS1 = featuresTS1[1]

featurests4 = ffdts.apply([date4])
featurests4_0 = featurests4[0]

print(np.arctanh((featurests4_0[-2:,openidx]-0.5)*2.0)+1.0)

[1.0151391 1.0026596]
